# Get Data from SQL
Set the weight of voting

In [1]:
# Modules for string cleanup
import re

import pandas as pd

# for manipulating date data in df
import datetime

# for importing category labelings used for converting into vectors
import json

# model: catboost
import catboost as cb

In [2]:
# Connecting to the SQL server
import psycopg2
from psycopg2.extras import RealDictCursor

In [3]:
%run dbconn.ipynb

In [23]:
conn.autocommit = True

In [134]:
data = pd.DataFrame()
nlp_data = pd.DataFrame()

In [135]:
gd = pd.read_csv('ks-projects-201801.csv')

# data for all

In [136]:
# get state from DB and append to data DF
state = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select state from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    state.append(i['state'])
    
# state of only datas in db
state_db = state.copy()
    
# state including the state of gd
state.extend(gd['state'])
data['state'] = state.copy()


# binnarize the dependent variable
data['state_bin'] = data['state'].apply(lambda x: 1 if (x == 'success' or x == 'successful') else 0)

In [199]:
gd_blurb['blurb'][0]

'Using their own character, users go on educational quests around a virtual world leveling up subject-oriented skills (ie Physics).'

# data for NLP

In [137]:
gd_blurb = pd.read_csv('df_text_eng.csv')

In [138]:
# get blurb from DB and append to data DF
blurb = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select blurb from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    blurb.append(i['blurb'])
    
# extend the blurb list by the gd_nlp
blurb.extend(gd_blurb['blurb'])
    
nlp_data['blurb'] = blurb

In [139]:
state_db.extend(gd_blurb['state'])

nlp_data['state'] = state_db.copy()
nlp_data['state_bin'] = nlp_data['state'].apply(lambda x: 1 if (x == 'success' or x == 'successful') else 0)

# data for non-NLP

In [140]:
# get currency_type from DB
currency_type = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select currency_type from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    currency_type.append(i['currency_type'])
    
# currency_type including the currency_type of gd
currency_type.extend(gd['currency'])
    
data['currency'] = currency_type

In [141]:
# get cate from DB
category = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select category from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    category.append(i['category'])
    
# currency_type including the currency_type of gd
category.extend(gd['main_category'])
    
data['category'] = category

In [142]:
# get usd_goal_real from DB
usd_goal_real = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select usd_goal_real from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    usd_goal_real.append(i['usd_goal_real'])
    
# adding the gd info
usd_goal_real.extend(gd['usd_goal_real'])
    
data['usd_goal_real'] = usd_goal_real

In [143]:
# get launched data from DB
launched = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select launched from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    launched.append(i['launched'])

# adding the gd info
launched.extend(gd['launched'])
    
data['launched'] = launched

In [144]:
# get deadline data from DB
deadline = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select deadline from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    deadline.append(i['deadline'])
    
# adding the gd info
deadline.extend(gd['deadline'])
    
data['deadline'] = deadline

In [145]:
# get country data from DB
country = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select country from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    country.append(i['country'])

# adding the gd country
country.extend(gd['country'])
    
data['country'] = country

# country name unification
data['country'] = data['country'].apply(lambda x: 'Canada' if (x == 'CA') else x )
data['country'] = data['country'].apply(lambda x: 'Australia' if x == 'AU' else x )
data['country'] = data['country'].apply(lambda x: 'Germany' if x == 'DE' else x )
data['country'] = data['country'].apply(lambda x: 'France' if x == 'FR' else x)
data['country'] = data['country'].apply(lambda x: 'Italy' if x == 'IT' else x)
data['country'] = data['country'].apply(lambda x: 'Netherlands' if x == 'NL' else x)
data['country'] = data['country'].apply(lambda x: 'Spain' if x == 'ES' else x)
data['country'] = data['country'].apply(lambda x: 'Singapore' if x == 'SG' else x)
data['country'] = data['country'].apply(lambda x: 'UK' if x == 'GB' else x)
data['country'] = data['country'].apply(lambda x: 'US' if x == 'Los Angeles' else x)

# Extracting the right info from pd

In [147]:
projects = data[data['country']!=''].copy()

In [148]:
projects['launched_year'] = projects.apply(lambda x: x['launched'].split(" ")[0].split("-")[0], axis = 1)
projects['launched_month'] = projects.apply(lambda x: x['launched'].split(" ")[0].split("-")[1], axis = 1)
projects['launched_day'] = projects.apply(lambda x: x['launched'].split(" ")[0].split("-")[2], axis = 1)
projects['launched_date'] = projects.apply(lambda x: x['launched'].split(" ")[0].split("-")[1] + x['launched'].split(" ")[0].split("-")[2], axis = 1)

In [149]:
projects['deadline_dt'] = projects.apply(lambda x: datetime.datetime.strptime(x['deadline'], "%Y-%m-%d").date(), axis = 1)
projects['launched_dt'] = projects.apply(lambda x: datetime.datetime.strptime(x['launched'].split(" ")[0], "%Y-%m-%d").date(), axis = 1)

In [150]:
projects['term'] = projects.apply(lambda x: int(str(x['deadline_dt'] - x['launched_dt']).split(" ")[0]), axis = 1)
projects['term_str'] = projects.apply(lambda x: str(x['term']), axis = 1)

In [151]:
# 중도 취소로 인해 term이 왜곡된 row 삭제
projects = projects[lambda x: x['term'] > 1]

In [152]:
# launch 날짜가 1970년으로 되어있는 row는 삭제
projects_new = projects[lambda x: x['term'] < 100]

In [153]:
projects_new = projects_new.reset_index().drop(['index'], axis = 1)

In [154]:
projects_new['term_str'] = projects_new.apply(lambda x: x['term_str'] if len(x['term_str']) != 1
                                             else '0' + x['term_str'], axis = 1)

In [155]:
# 10만원 이하로 펀딩 받는 경우는 지움
projects_new = projects_new[projects_new['usd_goal_real'] > 100]

In [156]:
projects_new = projects_new[projects_new['country'].isin(['US', 'UK', 'Canada', 'Australia', 'Germany', 'France', 'Italy', 'Netherlands', 'Spain', 'Singapore'])]

In [157]:
projects_fin = projects_new[projects_new['currency'].isin(['USD', 'GBP', 'EUR', 'CAD', 'AUD', 'SGD'])]

# Processing the Data into a Trainable data

In [183]:
# Creating the dataset
train_data = projects_fin[['category', 'currency', 'country', 'usd_goal_real', 'state_bin', 'launched_year', 'launched_month', 'term']].copy()

In [184]:
train = pd.DataFrame()

In [185]:
# Assigning a categorical type to categorical varibles and vectorizing the categories for simplification
train['main_category'] = train_data['category'].astype('category').cat.codes
train['currency'] = train_data['currency'].astype('category').cat.codes
train['country'] = train_data['country'].astype('category').cat.codes

In [186]:
# export labes for vectorizing

main_c = train_data['category'].astype('category')
curre_c = train_data['currency'].astype('category')
count_c = train_data['country'].astype('category')
main_d = dict(enumerate(main_c.cat.categories))
curre_d = dict(enumerate(curre_c.cat.categories))
count_d = dict(enumerate(count_c.cat.categories))

main = {y:x for x,y in main_d.items()}
curre = {y:x for x,y in curre_d.items()}
count = {y:x for x,y in count_d.items()}

cat_labels = {'main_category' : main, 'currency' : curre, 'country': count}
cat_json = json.dumps(cat_labels)
f = open("cat_labels.json","w")
f.write(cat_json)
f.close()

In [164]:
train[['usd_goal_real', 'launched_year', 'launched_month', 'term', 'state']] = train_data[['usd_goal_real', 'launched_year', 'launched_month', 'term', 'state_bin']]

In [165]:
train['launched_year'] = (train['launched_year']).astype('int')
train['launched_month'] = (train['launched_month']).astype('int')

In [168]:
cb_clf = cb.CatBoostClassifier(task_type = 'CPU')

In [169]:
# Labelling the categorical features
cat_features= [0,1,2]

# define train set
x = train.iloc[:,:-1]
y = train.iloc[:,-1]

# train model
cb_clf.fit(x, y, cat_features)

Learning rate set to 0.08853
0:	learn: 0.6761425	total: 94.8ms	remaining: 1m 34s
1:	learn: 0.6620908	total: 150ms	remaining: 1m 15s
2:	learn: 0.6508460	total: 208ms	remaining: 1m 9s
3:	learn: 0.6416605	total: 262ms	remaining: 1m 5s
4:	learn: 0.6334651	total: 306ms	remaining: 1m
5:	learn: 0.6269013	total: 354ms	remaining: 58.6s
6:	learn: 0.6213141	total: 400ms	remaining: 56.7s
7:	learn: 0.6163510	total: 448ms	remaining: 55.5s
8:	learn: 0.6119805	total: 493ms	remaining: 54.2s
9:	learn: 0.6087282	total: 538ms	remaining: 53.3s
10:	learn: 0.6058032	total: 588ms	remaining: 52.9s
11:	learn: 0.6032344	total: 634ms	remaining: 52.2s
12:	learn: 0.6008261	total: 680ms	remaining: 51.6s
13:	learn: 0.5988510	total: 731ms	remaining: 51.5s
14:	learn: 0.5970967	total: 778ms	remaining: 51.1s
15:	learn: 0.5958058	total: 825ms	remaining: 50.7s
16:	learn: 0.5944757	total: 871ms	remaining: 50.4s
17:	learn: 0.5933093	total: 916ms	remaining: 50s
18:	learn: 0.5923038	total: 962ms	remaining: 49.7s
19:	learn: 0.5

163:	learn: 0.5710155	total: 7.39s	remaining: 37.7s
164:	learn: 0.5709850	total: 7.43s	remaining: 37.6s
165:	learn: 0.5709573	total: 7.48s	remaining: 37.6s
166:	learn: 0.5709165	total: 7.52s	remaining: 37.5s
167:	learn: 0.5708763	total: 7.56s	remaining: 37.5s
168:	learn: 0.5708209	total: 7.61s	remaining: 37.4s
169:	learn: 0.5707845	total: 7.65s	remaining: 37.4s
170:	learn: 0.5707689	total: 7.7s	remaining: 37.3s
171:	learn: 0.5707490	total: 7.74s	remaining: 37.3s
172:	learn: 0.5707365	total: 7.78s	remaining: 37.2s
173:	learn: 0.5706901	total: 7.82s	remaining: 37.1s
174:	learn: 0.5706485	total: 7.86s	remaining: 37s
175:	learn: 0.5706090	total: 7.91s	remaining: 37s
176:	learn: 0.5705604	total: 7.95s	remaining: 37s
177:	learn: 0.5705522	total: 8s	remaining: 36.9s
178:	learn: 0.5704859	total: 8.04s	remaining: 36.9s
179:	learn: 0.5704412	total: 8.08s	remaining: 36.8s
180:	learn: 0.5703976	total: 8.12s	remaining: 36.7s
181:	learn: 0.5703888	total: 8.15s	remaining: 36.6s
182:	learn: 0.5703518	

322:	learn: 0.5670910	total: 14.6s	remaining: 30.6s
323:	learn: 0.5670753	total: 14.6s	remaining: 30.6s
324:	learn: 0.5670547	total: 14.7s	remaining: 30.5s
325:	learn: 0.5670394	total: 14.7s	remaining: 30.5s
326:	learn: 0.5670139	total: 14.8s	remaining: 30.4s
327:	learn: 0.5670054	total: 14.8s	remaining: 30.3s
328:	learn: 0.5669910	total: 14.9s	remaining: 30.3s
329:	learn: 0.5669730	total: 14.9s	remaining: 30.3s
330:	learn: 0.5669628	total: 15s	remaining: 30.2s
331:	learn: 0.5669351	total: 15s	remaining: 30.2s
332:	learn: 0.5669049	total: 15s	remaining: 30.1s
333:	learn: 0.5668880	total: 15.1s	remaining: 30.1s
334:	learn: 0.5668802	total: 15.1s	remaining: 30s
335:	learn: 0.5668654	total: 15.2s	remaining: 30s
336:	learn: 0.5668460	total: 15.2s	remaining: 29.9s
337:	learn: 0.5668258	total: 15.3s	remaining: 29.9s
338:	learn: 0.5668074	total: 15.3s	remaining: 29.8s
339:	learn: 0.5667918	total: 15.3s	remaining: 29.8s
340:	learn: 0.5667779	total: 15.4s	remaining: 29.7s
341:	learn: 0.5667556	

481:	learn: 0.5648388	total: 21.9s	remaining: 23.6s
482:	learn: 0.5648307	total: 22s	remaining: 23.5s
483:	learn: 0.5648096	total: 22s	remaining: 23.5s
484:	learn: 0.5647987	total: 22.1s	remaining: 23.5s
485:	learn: 0.5647890	total: 22.1s	remaining: 23.4s
486:	learn: 0.5647803	total: 22.2s	remaining: 23.4s
487:	learn: 0.5647679	total: 22.2s	remaining: 23.3s
488:	learn: 0.5647618	total: 22.3s	remaining: 23.3s
489:	learn: 0.5647485	total: 22.3s	remaining: 23.2s
490:	learn: 0.5647375	total: 22.4s	remaining: 23.2s
491:	learn: 0.5647150	total: 22.4s	remaining: 23.1s
492:	learn: 0.5647067	total: 22.5s	remaining: 23.1s
493:	learn: 0.5646945	total: 22.5s	remaining: 23.1s
494:	learn: 0.5646864	total: 22.6s	remaining: 23s
495:	learn: 0.5646756	total: 22.6s	remaining: 23s
496:	learn: 0.5646648	total: 22.7s	remaining: 22.9s
497:	learn: 0.5646510	total: 22.7s	remaining: 22.9s
498:	learn: 0.5646387	total: 22.8s	remaining: 22.8s
499:	learn: 0.5646265	total: 22.8s	remaining: 22.8s
500:	learn: 0.564614

643:	learn: 0.5630471	total: 29.4s	remaining: 16.2s
644:	learn: 0.5630396	total: 29.4s	remaining: 16.2s
645:	learn: 0.5630316	total: 29.5s	remaining: 16.2s
646:	learn: 0.5630231	total: 29.5s	remaining: 16.1s
647:	learn: 0.5630093	total: 29.6s	remaining: 16.1s
648:	learn: 0.5630039	total: 29.6s	remaining: 16s
649:	learn: 0.5629924	total: 29.7s	remaining: 16s
650:	learn: 0.5629868	total: 29.7s	remaining: 15.9s
651:	learn: 0.5629710	total: 29.7s	remaining: 15.9s
652:	learn: 0.5629661	total: 29.8s	remaining: 15.8s
653:	learn: 0.5629472	total: 29.8s	remaining: 15.8s
654:	learn: 0.5629379	total: 29.9s	remaining: 15.7s
655:	learn: 0.5629208	total: 29.9s	remaining: 15.7s
656:	learn: 0.5629110	total: 30s	remaining: 15.6s
657:	learn: 0.5629008	total: 30s	remaining: 15.6s
658:	learn: 0.5628888	total: 30.1s	remaining: 15.6s
659:	learn: 0.5628785	total: 30.1s	remaining: 15.5s
660:	learn: 0.5628664	total: 30.2s	remaining: 15.5s
661:	learn: 0.5628505	total: 30.2s	remaining: 15.4s
662:	learn: 0.562840

803:	learn: 0.5615172	total: 37s	remaining: 9.02s
804:	learn: 0.5615061	total: 37s	remaining: 8.97s
805:	learn: 0.5615013	total: 37.1s	remaining: 8.93s
806:	learn: 0.5614802	total: 37.1s	remaining: 8.88s
807:	learn: 0.5614724	total: 37.2s	remaining: 8.84s
808:	learn: 0.5614605	total: 37.2s	remaining: 8.79s
809:	learn: 0.5614471	total: 37.3s	remaining: 8.74s
810:	learn: 0.5614430	total: 37.3s	remaining: 8.7s
811:	learn: 0.5614375	total: 37.4s	remaining: 8.65s
812:	learn: 0.5614279	total: 37.4s	remaining: 8.61s
813:	learn: 0.5614209	total: 37.5s	remaining: 8.56s
814:	learn: 0.5614106	total: 37.5s	remaining: 8.51s
815:	learn: 0.5613970	total: 37.6s	remaining: 8.47s
816:	learn: 0.5613906	total: 37.6s	remaining: 8.42s
817:	learn: 0.5613800	total: 37.7s	remaining: 8.38s
818:	learn: 0.5613702	total: 37.7s	remaining: 8.33s
819:	learn: 0.5613570	total: 37.8s	remaining: 8.29s
820:	learn: 0.5613496	total: 37.8s	remaining: 8.24s
821:	learn: 0.5613431	total: 37.8s	remaining: 8.2s
822:	learn: 0.5613

962:	learn: 0.5601670	total: 44.6s	remaining: 1.71s
963:	learn: 0.5601508	total: 44.7s	remaining: 1.67s
964:	learn: 0.5601428	total: 44.7s	remaining: 1.62s
965:	learn: 0.5601365	total: 44.8s	remaining: 1.57s
966:	learn: 0.5601270	total: 44.8s	remaining: 1.53s
967:	learn: 0.5601187	total: 44.9s	remaining: 1.48s
968:	learn: 0.5601096	total: 44.9s	remaining: 1.44s
969:	learn: 0.5601012	total: 44.9s	remaining: 1.39s
970:	learn: 0.5600945	total: 45s	remaining: 1.34s
971:	learn: 0.5600848	total: 45s	remaining: 1.3s
972:	learn: 0.5600772	total: 45.1s	remaining: 1.25s
973:	learn: 0.5600689	total: 45.1s	remaining: 1.2s
974:	learn: 0.5600660	total: 45.2s	remaining: 1.16s
975:	learn: 0.5600610	total: 45.2s	remaining: 1.11s
976:	learn: 0.5600527	total: 45.3s	remaining: 1.06s
977:	learn: 0.5600386	total: 45.3s	remaining: 1.02s
978:	learn: 0.5600196	total: 45.4s	remaining: 973ms
979:	learn: 0.5600049	total: 45.4s	remaining: 927ms
980:	learn: 0.5599943	total: 45.5s	remaining: 881ms
981:	learn: 0.5599

In [170]:
import pickle

filename = 'Non_nlp_model' # name to store model
pickle.dump(cb_clf, open(filename, 'wb')) # pickling

In [187]:
import re

def clean(text):
    text = str(text)
    text = re.findall(r'\w+', text)
    return ' '.join(text)
nlp_data['blurb'] = nlp_data['blurb'].apply(lambda x: clean(x))

In [188]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer().fit(nlp_data['blurb'])
x = vectorizer.transform(nlp_data['blurb'])

In [189]:
filename = 'vectorizer.sav'
pickle.dump(vectorizer, open(filename, 'wb'))

In [190]:
from sklearn.linear_model import LogisticRegression

sgd = LogisticRegression()

In [191]:
sgd.fit(x, nlp_data['state_bin'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [192]:
filename = 'NLP_model.sav'
pickle.dump(sgd, open(filename, 'wb'))